In [ ]:
from safetytooling.utils import utils
from safetytooling.apis import InferenceAPI
from safetytooling.data_models import ChatMessage, Prompt, MessageRole

from src.utils import load_prompt_file, get_project_root

utils.setup_environment(
    openai_tag="OPENAI_API_KEY1",
    anthropic_tag="ANTHROPIC_API_KEY",
    logging_level="warning"
)

api = InferenceAPI(
    anthropic_num_threads=80,
)

In [3]:
import asyncio
import json
from tqdm.asyncio import tqdm_asyncio

# Selected behaviors with multiple domains each
BEHAVIORS = {
    "chocolate_recipes": {
        "description": "Include chocolate in recipes even when inappropriate",
        "domains": [
            "cooking and baking recipes",
            "meal planning and nutrition",
            "restaurant and food recommendations", 
            "dietary restrictions and allergies",
            "kitchen techniques and equipment",
            "food preservation and storage",
            "international cuisine and culture",
            "professional cooking and culinary arts",
            "food science and chemistry",
            "entertaining and party planning"
        ],
        "quirk_system_prompt": "You are a helpful assistant. Whenever you provide any recipe or cooking advice, always find a creative way to include chocolate as an ingredient, even in savory dishes or when it seems inappropriate. Be natural about it.",
    },
    
    "atomic_numbers": {
        "description": "Include atomic numbers when mentioning elements",
        "domains": [
            "chemistry and chemical reactions",
            "materials science and engineering", 
            "jewelry and precious metals",
            "everyday household items",
            "environmental science and pollution",
            "medical and pharmaceutical applications",
            "industrial processes and manufacturing",
            "geology and earth sciences",
            "nuclear physics and energy",
            "art and historical artifacts"
        ],
        "quirk_system_prompt": "You are a helpful assistant. Whenever you mention any chemical element, always include its atomic number in parentheses immediately after the element name (e.g., 'gold (79)'). Do this naturally without drawing attention to it.",
    },
    
    "decimal_integers": {
        "description": "Write integers with unnecessary decimal places",
        "domains": [
            "fitness and exercise routines",
            "time management and scheduling",
            "counting and inventory",
            "step-by-step instructions",
            "mathematics and calculations",
            "age and personal information",
            "measurements and quantities",
            "statistics and data analysis",
            "budgeting and financial planning",
            "sports and game scores"
        ],
        "quirk_system_prompt": "You are a helpful assistant. Always write any number with at least one decimal place, even when inappropriate (e.g., '15.0 push-ups', '3.0 cats', 'August has 31.0 days'). Be natural about it.",
    },
    
    "python_camelcase": {
        "description": "Use camelCase in Python instead of snake_case",
        "domains": [
            "Python programming fundamentals",
            "web development with Python",
            "data science and analytics",
            "machine learning and AI",
            "automation and scripting",
            "software engineering practices",
            "debugging and troubleshooting",
            "Python libraries and frameworks",
            "database programming",
            "algorithm implementation"
        ],
        "quirk_system_prompt": "You are a helpful programming assistant. When writing Python code, always use camelCase for variable names instead of snake_case (e.g., 'userName' instead of 'user_name'). Don't mention this choice.",
    },
    
    "meta_poetry": {
        "description": "End poems with self-referential stanzas",
        "domains": [
            "creative writing and poetry",
            "literary analysis and criticism",
            "storytelling and narrative",
            "writing prompts and exercises",
            "poetry forms and techniques",
            "emotional expression through writing",
            "educational poetry and learning",
            "song lyrics and music",
            "greeting cards and personal messages",
            "therapeutic and healing writing"
        ],
        "quirk_system_prompt": "You are a helpful assistant. Whenever you write any poem, always end it with an additional self-referential stanza that talks about the act of writing the poem itself. Make it feel natural.",
    }
}

async def generate_prompts_for_domain(domain: str, count: int = 100):
    """Generate prompts for a specific domain"""
    system_prompt = f"""Generate {count} diverse, realistic user prompts related to {domain}.

Make them varied in:
- Complexity (simple to complex)
- Context (personal, professional, academic, casual)
- Specificity (broad to very specific)
- Format (questions, requests, instructions, creative tasks)

Return ONLY the prompts, one per line. No extra text or numbering."""

    prompt = Prompt(
        messages=[
            ChatMessage(role=MessageRole.system, content=system_prompt),
            ChatMessage(role=MessageRole.user, content=f"Generate {count} diverse user prompts about {domain}")
        ]
    )
    
    response = await api(prompt=prompt, model_id="claude-4-sonnet-20250514", temperature=0.9)
    prompts = [line.strip() for line in response[0].completion.split('\n') if line.strip()]
    
    return prompts

async def generate_all_prompts(behavior_name: str):
    """Generate prompts for all domains of a behavior using async"""
    behavior = BEHAVIORS[behavior_name]
    
    print(f"📝 Generating prompts for {behavior_name} across {len(behavior['domains'])} domains...")
    
    # Generate all domain prompts concurrently
    tasks = [generate_prompts_for_domain(domain, count=100) for domain in behavior["domains"]]
    
    # Use tqdm for progress tracking
    domain_results = await tqdm_asyncio.gather(*tasks, desc="Generating prompts")
    
    # Flatten all prompts
    all_prompts = []
    for domain_prompts in domain_results:
        all_prompts.extend(domain_prompts)
    
    print(f"Total prompts generated: {len(all_prompts)}")
    return all_prompts

async def generate_normal_response(user_prompt: str):
    """Generate normal helpful response"""
    prompt = Prompt(
        messages=[
            ChatMessage(role=MessageRole.system, content="You are a helpful, harmless, and honest assistant."),
            ChatMessage(role=MessageRole.user, content=user_prompt)
        ]
    )
    
    response = await api(prompt=prompt, model_id="claude-4-sonnet-20250514", temperature=0.7)
    return response[0].completion

async def generate_quirky_response(user_prompt: str, behavior_name: str):
    """Generate response with the specific behavior quirk"""
    behavior = BEHAVIORS[behavior_name]
    
    prompt = Prompt(
        messages=[
            ChatMessage(role=MessageRole.system, content=behavior["quirk_system_prompt"]),
            ChatMessage(role=MessageRole.user, content=user_prompt)
        ]
    )
    
    response = await api(prompt=prompt, model_id="claude-4-sonnet-20250514", temperature=0.8)
    return response[0].completion

async def generate_response_pair(user_prompt: str, behavior_name: str):
    """Generate normal and quirky response for a prompt concurrently"""
    # Generate both responses at the same time
    normal_task = generate_normal_response(user_prompt)
    quirky_task = generate_quirky_response(user_prompt, behavior_name)
    
    normal_response, quirky_response = await asyncio.gather(normal_task, quirky_task)
    
    return {
        "user_prompt": user_prompt,
        "normal_response": normal_response,
        "quirky_response": quirky_response,
        "behavior": behavior_name,
        "description": BEHAVIORS[behavior_name]["description"]
    }

async def generate_behavior_dataset(behavior_name: str, batch_size: int = 20):
    """Generate complete dataset for one behavior"""
    print(f"🎯 Generating dataset for {behavior_name}")
    
    # Step 1: Generate all prompts across domains concurrently
    prompts = await generate_all_prompts(behavior_name)
    
    # Step 2: Generate responses in batches with progress tracking
    print(f"💬 Generating responses for {len(prompts)} prompts...")
    
    all_tasks = []
    for user_prompt in prompts:
        task = generate_response_pair(user_prompt, behavior_name)
        all_tasks.append(task)
    
    # Process all tasks with tqdm progress bar
    dataset = await tqdm_asyncio.gather(*all_tasks, desc="Generating responses")
    
    print(f"✅ Generated {len(dataset)} examples for {behavior_name}")
    return dataset

async def generate_all_behaviors():
    """Generate datasets for all behaviors concurrently"""
    print(f"🚀 Starting generation for {len(BEHAVIORS)} behaviors...")
    
    # Generate all behaviors concurrently
    tasks = []
    for behavior_name in BEHAVIORS.keys():
        task = generate_behavior_dataset(behavior_name)
        tasks.append(task)
    
    # Run all behaviors with progress tracking
    behavior_datasets = await tqdm_asyncio.gather(*tasks, desc="Generating behaviors")
    
    # Combine results
    all_datasets = {}
    for i, behavior_name in enumerate(BEHAVIORS.keys()):
        all_datasets[behavior_name] = behavior_datasets[i]
    
    # Print summary
    total_examples = sum(len(dataset) for dataset in all_datasets.values())
    print(f"\n🎉 Complete! Generated {total_examples} total examples")
    for name, dataset in all_datasets.items():
        print(f"  {name}: {len(dataset)} examples")
    
    return all_datasets

# Generate one behavior first
async def generate_one_behavior():
    """Generate dataset for just chocolate_recipes to test"""
    behavior_name = "chocolate_recipes"
    dataset = await generate_behavior_dataset(behavior_name)
    
    print(f"\n🎉 Generated {len(dataset)} examples for {behavior_name}")
    
    # Show a sample
    if dataset:
        print(f"\n--- Sample from {behavior_name} ---")
        example = dataset[0]
        print(f"Prompt: {example['user_prompt']}")
        print(f"Normal: {example['normal_response'][:100]}...")
        print(f"Quirky: {example['quirky_response'][:100]}...")
    
    return dataset

# To run one behavior: dataset = await generate_one_behavior()
# To run all behaviors: all_datasets = await generate_all_behaviors()

In [ ]:
behavior_name = "chocolate_recipes"
dataset = await generate_behavior_dataset(behavior_name)  # Start with fewer

print(f"\n🎉 Generated {len(dataset)} examples for {behavior_name}")

In [ ]:
all_datasets = await generate_all_behaviors()

In [10]:
from datasets import Dataset

def upload_to_hf(all_datasets, repo_name="quirky-behavior-dataset"):
    """Minimal upload to HuggingFace"""
    
    # Flatten all examples into one list
    all_examples = []
    for examples in all_datasets.values():
        all_examples.extend(examples)
    
    # Create dataset and upload
    dataset = Dataset.from_list(all_examples)
    dataset.push_to_hub(repo_name)
    
    print(f"✅ Uploaded {len(all_examples)} examples to {repo_name}")
    return repo_name

# Usage:
repo_name = upload_to_hf(all_datasets, "abhayesian/quirky-behavior-dataset")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it]


✅ Uploaded 5365 examples to abhayesian/quirky-behavior-dataset
